In [1]:
# importing the dependencies. 
import pandas as pd
import numpy as np
import h5py
import obspy
from obspy.signal.filter import envelope
from tqdm import tqdm
import tsfel
from tsfel import time_series_features_extractor

import warnings
import argparse
import sys
import os



# Get the absolute path of the directory two levels up
two_levels_up = os.path.abspath(os.path.join(os.getcwd(), "../.."))

# Append the 'src' directory located two levels up to the system path
sys.path.append(os.path.join(two_levels_up, 'src'))

from utils import apply_cosine_taper
from utils import butterworth_filter
from utils import resample_array

# Ignore all warnings
warnings.filterwarnings("ignore")

# displaying all columns from pandas dataframe
pd.set_option('display.max_columns', None)


from glob import glob

In [2]:
# Load dataset
new_su_cat = pd.read_csv(glob('../../data/additional_surface_events_good_snr.csv')[0], index_col=0)

# Constants
TAPER_AMOUNT = 10
LOW_FREQ, HIGH_FREQ = 1, 10
ORIGINAL_SR, NEW_SR = 100, 50
NUM_CORNERS = 4
START, END = 50, 100
ARRIVAL = 7000

cfg_file = tsfel.get_features_by_domain()
# Initialize lists for storing results
data, evids = [], []

# Process each event
for _, row in tqdm(new_su_cat.iterrows(), total=len(new_su_cat)):
    try:
        new_file_path = row['file_path'].replace('HN', 'HZ')
        evid = f"{row['event_id']}_{row['station_network_code']}.{row['station_code']}"

        # Read and preprocess waveform
        stream = obspy.read('../' + new_file_path)
        stream.detrend('linear')
        stream.resample(ORIGINAL_SR)

        tr = stream[0].data[int(ARRIVAL - START * ORIGINAL_SR) : int(ARRIVAL + END * ORIGINAL_SR)]
        data.append(tr)
        evids.append(evid)
    except Exception as e:
        print(f"Skipping event {row['event_id']} due to error: {e}")

# Convert to NumPy array
data = np.array(data)
print(f"Data shape: {data.shape}")

# Signal processing
tapered_data = apply_cosine_taper(data, taper_percent=TAPER_AMOUNT)
filtered_data = np.array(butterworth_filter(tapered_data, LOW_FREQ, HIGH_FREQ, ORIGINAL_SR, NUM_CORNERS, 'bandpass'))
normalized_data = filtered_data / np.max(abs(filtered_data), axis=1, keepdims=True)
resampled_data = np.array([resample_array(arr, ORIGINAL_SR, NEW_SR) for arr in normalized_data])

# Feature extraction
feature_list = []
for i in tqdm(range(len(resampled_data))):
    try:
        features = time_series_features_extractor(cfg_file, resampled_data[i], fs= NEW_SR, verbose = 0)
        features['new_event_id'] = evids[i]
        feature_list.append(features)
    except Exception as e:
        print(f"Feature extraction failed for {evids[i]}: {e}")

# Combine features into a DataFrame
features_df = pd.concat(feature_list, ignore_index=True)
print(f"Feature DataFrame shape: {features_df.shape}")

low_freq = str(LOW_FREQ).replace('.','')
features_df.to_csv(f'../../extracted_features/tsfel_features_new_surface event_P_{START}_{END}_F_'+low_freq+f'_{HIGH_FREQ}_{NEW_SR}_part_1.csv')



  5%|▍         | 320/6517 [00:05<01:56, 52.97it/s]

Skipping event pnsn1080683 due to error: [Errno 2] No such file or directory: '../../data/surface_event_waveforms/CC.VALT..BHZ__20100711T001407Z__20100711T001707Z.mseed'


  8%|▊         | 522/6517 [00:08<01:37, 61.23it/s]

Skipping event pnsn1115258 due to error: [Errno 2] No such file or directory: '../../data/surface_event_waveforms/CC.OBSR..BHZ__20110919T204456Z__20110919T204756Z.mseed'


 18%|█▊        | 1179/6517 [00:20<01:21, 65.61it/s]

...

Skipping event pnsn3123648 due to error: [Errno 2] No such file or directory: '../../data/surface_event_waveforms/UW.LON..HHZ__20200415T214452Z__20200415T214752Z.mseed'


100%|██████████| 6517/6517 [01:51<00:00, 58.39it/s]


Data shape: (6512, 15000)


100%|██████████| 6512/6512 [07:48<00:00, 13.90it/s]


Feature DataFrame shape: (6512, 391)


,0_FFT mean coefficient_0,0_FFT mean coefficient_1,0_FFT mean coefficient_2,0_FFT mean coefficient_3,0_FFT mean coefficient_4,0_FFT mean coefficient_5,0_FFT mean coefficient_6,0_FFT mean coefficient_7,0_FFT mean coefficient_8,0_FFT mean coefficient_9,0_FFT mean coefficient_10,0_FFT mean coefficient_11,0_FFT mean coefficient_12,0_FFT mean coefficient_13,0_FFT mean coefficient_14,0_FFT mean coefficient_15,0_FFT mean coefficient_16,0_FFT mean coefficient_17,0_FFT mean coefficient_18,0_FFT mean coefficient_19,0_FFT mean coefficient_20,0_FFT mean coefficient_21,0_FFT mean coefficient_22,0_FFT mean coefficient_23,0_FFT mean coefficient_24,0_FFT mean coefficient_25,0_FFT mean coefficient_26,0_FFT mean coefficient_27,0_FFT mean coefficient_28,0_FFT mean coefficient_29,0_FFT mean coefficient_30,0_FFT mean coefficient_31,0_FFT mean coefficient_32,0_FFT mean coefficient_33,0_FFT mean coefficient_34,0_FFT mean coefficient_35,0_FFT mean coefficient_36,0_FFT mean coefficient_37,0_FFT mean coefficient_38,0_FFT mean coefficient_39,0_FFT mean coefficient_40,0_FFT mean coefficient_41,0_FFT mean coefficient_42,0_FFT mean coefficient_43,0_FFT mean coefficient_44,0_FFT mean coefficient_45,0_FFT mean coefficient_46,0_FFT mean coefficient_47,0_FFT mean coefficient_48,0_FFT mean coefficient_49,0_FFT mean coefficient_50,0_FFT mean coefficient_51,0_FFT mean coefficient_52,0_FFT mean coefficient_53,0_FFT mean coefficient_54,0_FFT mean coefficient_55,0_FFT mean coefficient_56,0_FFT mean coefficient_57,0_FFT mean coefficient_58,0_FFT mean coefficient_59,0_FFT mean coefficient_60,0_FFT mean coefficient_61,0_FFT mean coefficient_62,0_FFT mean coefficient_63,0_FFT mean coefficient_64,0_FFT mean coefficient_65,0_FFT mean coefficient_66,0_FFT mean coefficient_67,0_FFT mean coefficient_68,0_FFT mean coefficient_69,0_FFT mean coefficient_70,0_FFT mean coefficient_71,0_FFT mean coefficient_72,0_FFT mean coefficient_73,0_FFT mean coefficient_74,0_FFT mean coefficient_75,0_FFT mean coefficient_76,0_FFT mean coefficient_77,0_FFT mean coefficient_78,0_FFT mean coefficient_79,0_FFT mean coefficient_80,0_FFT mean coefficient_81,0_FFT mean coefficient_82,0_FFT mean coefficient_83,0_FFT mean coefficient_84,0_FFT mean coefficient_85,0_FFT mean coefficient_86,0_FFT mean coefficient_87,0_FFT mean coefficient_88,0_FFT mean coefficient_89,0_FFT mean coefficient_90,0_FFT mean coefficient_91,0_FFT mean coefficient_92,0_FFT mean coefficient_93,0_FFT mean coefficient_94,0_FFT mean coefficient_95,0_FFT mean coefficient_96,0_FFT mean coefficient_97,0_FFT mean coefficient_98,0_FFT mean coefficient_99,0_FFT mean coefficient_100,0_FFT mean coefficient_101,0_FFT mean coefficient_102,0_FFT mean coefficient_103,0_FFT mean coefficient_104,0_FFT mean coefficient_105,0_FFT mean coefficient_106,0_FFT mean coefficient_107,0_FFT mean coefficient_108,0_FFT mean coefficient_109,0_FFT mean coefficient_110,0_FFT mean coefficient_111,0_FFT mean coefficient_112,0_FFT mean coefficient_113,0_FFT mean coefficient_114,0_FFT mean coefficient_115,0_FFT mean coefficient_116,0_FFT mean coefficient_117,0_FFT mean coefficient_118,0_FFT mean coefficient_119,0_FFT mean coefficient_120,0_FFT mean coefficient_121,0_FFT mean coefficient_122,0_FFT mean coefficient_123,0_FFT mean coefficient_124,0_FFT mean coefficient_125,0_FFT mean coefficient_126,0_FFT mean coefficient_127,0_FFT mean coefficient_128,0_FFT mean coefficient_129,0_FFT mean coefficient_130,0_FFT mean coefficient_131,0_FFT mean coefficient_132,0_FFT mean coefficient_133,0_FFT mean coefficient_134,0_FFT mean coefficient_135,0_FFT mean coefficient_136,0_FFT mean coefficient_137,0_FFT mean coefficient_138,0_FFT mean coefficient_139,0_FFT mean coefficient_140,0_FFT mean coefficient_141,0_FFT mean coefficient_142,0_FFT mean coefficient_143,0_FFT mean coefficient_144,0_FFT mean coefficient_145,0_FFT mean coefficient_146,0_FFT mean coefficient_147,0_FFT mean coefficient_148,0_FFT mean coefficient_149,0_FFT mean coefficient_150,0_FFT mean coefficient_151,0_FFT